<a href="https://colab.research.google.com/github/PTC-Education/PTC-Education.github.io/blob/master/Motion_Simulation_GIF_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up Onshape Client

In [1]:
!pip install onshape-client
from onshape_client.client import Client
import json
base = 'https://cad.onshape.com' # change this if you're using a document in an enterprise (i.e. "https://ptc.onshape.com")

     |████████████████████████████████| 2.1 MB 16.3 MB/s 
     |████████████████████████████████| 204 kB 50.6 MB/s 
     |████████████████████████████████| 108 kB 49.8 MB/s 
     |████████████████████████████████| 546 kB 34.3 MB/s 


In [2]:
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  execfile(fn)

client = Client(configuration={"base_url": base,
                               "access_key": access,
                               "secret_key": secret})
print('client configured')

Saving colabkeys.py to colabkeys.py
client configured


# Define Motion Function

In [ ]:
def moveMate(rot):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/matevalues'

  # https://cad.onshape.com/documents/cc9c5cb9912e286856491ca1/w/46ca1ebc66132096757402de/e/b23d54c6d17e2dc553e99171
  did = 'cc9c5cb9912e286856491ca1'
  wid = '46ca1ebc66132096757402de'
  eid = 'b23d54c6d17e2dc553e99171'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
              'Content-Type': 'application/vnd.onshape.v2+json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  # The command below prints the entire JSON response from Onshape
  parsed['mateValues'][0]['rotationZ'] = rot

  response = client.api_client.request('POST', url=base + fixed_url, query_params=params, headers=headers, body=parsed)

moveMate(1)

## Inverse Kinematics for 5D robot

In [ ]:
import math
import numpy
DegToRad = math.pi/180
RadToDeg = 180/math.pi

FloorToFirstJoint = 2.61*0.0254
UpperArmLength = 5.74*0.0254
ForeArmLength = 7.23*0.0254
GripperLength = 4.43*0.0254

def GetJointAngles(TCPx,TCPy,TCPz,WristAngleIn):
  BaseAngle = math.atan2(TCPx,TCPy)

  TCPq = math.sqrt(TCPx**2 + TCPy**2)
  TCPp = TCPz - FloorToFirstJoint

  WristQ = TCPq - GripperLength*math.cos(WristAngleIn*DegToRad)
  WristP = TCPp + GripperLength*math.sin(WristAngleIn*DegToRad)

  D2 = math.sqrt(WristP**2 + WristQ**2)
  print(WristQ,WristP)

  ElbowAngle = math.acos((WristP**2 + WristQ**2 - UpperArmLength**2 - ForeArmLength**2)/(2*UpperArmLength*ForeArmLength))

  a1 = math.atan2(ForeArmLength*math.sin(ElbowAngle),UpperArmLength + ForeArmLength*math.cos(ElbowAngle))
  a2 = math.atan2(WristP,WristQ)
  ShoulderAngle = a1 + a2

  WristAngleOut = ShoulderAngle - ElbowAngle + WristAngleIn*DegToRad + math.pi/2

  print(numpy.dot([BaseAngle, ShoulderAngle, ElbowAngle, WristAngleOut],RadToDeg))
  return [BaseAngle, ShoulderAngle, ElbowAngle, WristAngleOut]

## Set Mate Values of robot arm assembly

In [10]:
def getMateValues():
  RobotData = []
  RobotData = [0 for i in range(6)]
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/matevalues'

  # https://cad.onshape.com/documents/4bda16c648566259ea1b4e4c/w/c299b9fc994574c2637e871d/e/2f52bf4870f9d7ddc900b4de
  did = '4bda16c648566259ea1b4e4c'
  wid = 'c299b9fc994574c2637e871d'
  eid = '2f52bf4870f9d7ddc900b4de'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  fullResponse = json.loads(response.data)

  for i in range(len(fullResponse["mateValues"])):
    if fullResponse['mateValues'][i]['mateName'] == "Base":
      RobotData[0] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,500,2500))
    elif fullResponse['mateValues'][i]['mateName'] == "Shoulder":
      RobotData[1] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,500,2500))
    elif fullResponse['mateValues'][i]['mateName'] == "Elbow":
      RobotData[2] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,500,2500))-70
    elif fullResponse['mateValues'][i]['mateName'] == "Wrist":
      RobotData[3] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,2500,500))+50
    elif fullResponse['mateValues'][i]['mateName'] == "Hand":
      RobotData[4] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,500,2500))
    elif fullResponse['mateValues'][i]['mateName'] == "Gripper":
      RobotData[5] = int(translate(fullResponse['mateValues'][i]['rotationZ'],0,3.14,500,2500))
  
  return RobotData

def setMateValues(baseAngle,shoulderAngle,elbowAngle,wristAngle,handAngle,gripperAngle):
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/matevalues'

  # https://cad.onshape.com/documents/4bda16c648566259ea1b4e4c/w/c299b9fc994574c2637e871d/e/2f52bf4870f9d7ddc900b4de
  did = '4bda16c648566259ea1b4e4c'
  wid = 'c299b9fc994574c2637e871d'
  eid = '2f52bf4870f9d7ddc900b4de'

  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  fullResponse = json.loads(response.data)
  
  for i in range(len(fullResponse["mateValues"])):
    if fullResponse['mateValues'][i]['mateName'] == "Base":
      print(fullResponse['mateValues'][i])
      fullResponse['mateValues'][i]['rotationZ'] = baseAngle
      print(fullResponse['mateValues'][i])

  method = 'POST'

  params = {}
  payload = fullResponse
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}
  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  for i in range(len(fullResponse["mateValues"])):
    if fullResponse['mateValues'][i]['mateName'] == "Shoulder":
      fullResponse['mateValues'][i]['rotationZ'] = shoulderAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Elbow":
      fullResponse['mateValues'][i]['rotationZ'] = elbowAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Wrist":
      fullResponse['mateValues'][i]['rotationZ'] = wristAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Hand":
      fullResponse['mateValues'][i]['rotationZ'] = handAngle
    elif fullResponse['mateValues'][i]['mateName'] == "Gripper":
      fullResponse['mateValues'][i]['rotationZ'] = gripperAngle

  method = 'POST'

  params = {}
  payload = fullResponse
  headers = {'Accept': 'application/vnd.onshape.v2+json',
            'Content-Type': 'application/vnd.onshape.v2+json'}
  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  # The command below prints the entire JSON response from Onshape
  print(response.status)

def translate(value, leftMin, leftMax, rightMin, rightMax):
  # Figure out how 'wide' each range is
  leftSpan = leftMax - leftMin
  rightSpan = rightMax - rightMin

  # Convert the left range into a 0-1 range (float)
  valueScaled = float(value - leftMin) / float(leftSpan)

  # Convert the 0-1 range into a value in the right range.
  return rightMin + (valueScaled * rightSpan)

## Generate array of positions

In [31]:
import math
posArray = []
for i in range(15):
  posArray.append(math.pi/(i+2))

print(posArray)
FullArray = []

for i in range(6):
  FullArray.append(posArray)
print(len(FullArray[0]))
print(FullArray[0][0],FullArray[5][0])

[1.5707963267948966, 1.0471975511965976, 0.7853981633974483, 0.6283185307179586, 0.5235987755982988, 0.4487989505128276, 0.39269908169872414, 0.3490658503988659, 0.3141592653589793, 0.28559933214452665, 0.2617993877991494, 0.241660973353061, 0.2243994752564138, 0.20943951023931953, 0.19634954084936207]
15
1.5707963267948966 1.5707963267948966


## Get the names views of the element if there are any

In [ ]:
fixed_url = '/api/assemblies/d/did/e/eid/namedViews'

# https://cad.onshape.com/documents/cc9c5cb9912e286856491ca1/w/46ca1ebc66132096757402de/e/b23d54c6d17e2dc553e99171
did = 'cc9c5cb9912e286856491ca1'
wid = '46ca1ebc66132096757402de'
eid = 'b23d54c6d17e2dc553e99171'

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json',
            'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
print(response.data)

parsed = json.loads(response.data)
# The command below prints the entire JSON response from Onshape

# Get shaded view of model

In [34]:
import base64
def shadedViews():
  fixed_url = '/api/assemblies/d/did/w/wid/e/eid/shadedviews'

  # https://cad.onshape.com/documents/4bda16c648566259ea1b4e4c/w/c299b9fc994574c2637e871d/e/2f52bf4870f9d7ddc900b4de
  did = '4bda16c648566259ea1b4e4c'
  wid = 'c299b9fc994574c2637e871d'
  eid = '2f52bf4870f9d7ddc900b4de'

  method = 'GET'

  ## View Matrix below is roughly isometric
  params = {'viewMatrix':'0.612,0.612,0,0,-0.354,0.354,0.707,0,0.707,-0.707,0.707,0',
            'edges':'show',
            'outputHeight':500,
            'outputWidth':500,
            'pixelSize':0.0025}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json',
              'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)

  imgdata = base64.b64decode(parsed['images'][0])
  
  return imgdata
  # with open(filename, 'wb') as f:
  #     f.write(imgdata)

In [ ]:
import time
import imageio
import io
images = []
for i in range(len(FullArray[0])):
  setMateValues(FullArray[0][i],FullArray[1][i],FullArray[2][i],FullArray[3][i],FullArray[4][i],FullArray[5][i])
  images.append(imageio.imread(io.BytesIO(shadedViews())))

imageio.mimsave('OnshapeGif.gif', images)


{'jsonType': 'Revolute', 'rotationZ': 0.20917128323578119, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
{'jsonType': 'Revolute', 'rotationZ': 1.5707963267948966, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
200
{'jsonType': 'Revolute', 'rotationZ': 0.38534057863474436, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
{'jsonType': 'Revolute', 'rotationZ': 1.0471975511965976, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
200
{'jsonType': 'Revolute', 'rotationZ': 1.1905451201019752, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
{'jsonType': 'Revolute', 'rotationZ': 0.7853981633974483, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
200
{'jsonType': 'Revolute', 'rotationZ': 0.946720444978882, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
{'jsonType': 'Revolute', 'rotationZ': 0.6283185307179586, 'mateName': 'Base', 'featureId': 'MP0JnvCPgEfKeP2HM'}
200
{'jsonType': 'Revolute', 'rotationZ': 0.7482352249790749, 'mateName': 'Base', 'featureI